# ***DISTRIBUCIÓN DE SEGMENTOS***

In [26]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df

# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# *Data import*

In [118]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
# Imprimir la ruta relativa
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

### *Creación de los df por cada archivo json*

In [119]:
dataframes = {} # Crear un diccionario para almacenar los DataFrames
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

In [120]:
# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres para obtener solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Imprimir la lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_sep_2023_pachuca', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2023_pachuca', 'df_grupos_edad_pachuca', 'df_usuarios', 'df_segmentos_per', 'df_percent', 'df_percents']


_________
# *Distribución*
### Selección de datos

In [121]:
# Crea los DataFrames para cada archivo con las columnas necesarias
segment_df_alfa_q_oct_2022_pachuca = df_alfa_q_oct_2022_pachuca[['id','categoria']]
segment_df_alfa_q_nov_2022_pachuca = df_alfa_q_nov_2022_pachuca[['id','categoria']]
segment_df_alfa_q_feb_2023_pachuca = df_alfa_q_feb_2023_pachuca[['id','categoria']]
segment_df_alfa_q_mar_2023_pachuca = df_alfa_q_mar_2023_pachuca[['id','categoria']]
segment_df_alfa_q_jun_2023_pachuca = df_alfa_q_jun_2023_pachuca[['id','categoria']]
segment_df_alfa_q_jul_2023_pachuca = df_alfa_q_jul_2023_pachuca[['id','categoria']]
segment_df_alfa_q_sep_2023_pachuca = df_alfa_q_sep_2023_pachuca[['id','categoria']] # Vacía

### Conteo único de datos "categoria"

In [166]:
# Lista de DataFrames y sus nombres correspondientes
dataframes_list = [
    segment_df_alfa_q_oct_2022_pachuca,
    segment_df_alfa_q_nov_2022_pachuca,
    segment_df_alfa_q_feb_2023_pachuca,
    segment_df_alfa_q_mar_2023_pachuca,
    segment_df_alfa_q_jun_2023_pachuca,
    segment_df_alfa_q_jul_2023_pachuca
]

# Inicializar un diccionario para almacenar el conteo total de valores únicos
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])

# Imprimir el DataFrame con el conteo total
conteo_total_df

,categoria,conteo_total
0,S,1824
1,D1,782
2,C3,617
3,B1,686
4,B2,645
5,B3,571
6,D2,506
7,C2,516
8,D3,395
9,A3,399


In [167]:
# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)
# Crear una nueva fila como un diccionario de Python
nueva_fila = {'conteo_total': 0, 'segmento': 'E', 'clase': '3'}
# Agregar la nueva fila al DataFrame
conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)

conteo_total_df

C:\Users\yoe11\AppData\Local\Temp\ipykernel_5232\3569535127.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)


,conteo_total,segmento,clase
0,1824,S,1
1,782,D,1
2,617,C,3
3,686,B,1
4,645,B,2
5,571,B,3
6,506,D,2
7,516,C,2
8,395,D,3
9,399,A,3


In [168]:
#segmentos_dist = conteo_total_df.groupby(['segmento', 'clase']) # Agrupar
segmentos_dist = conteo_total_df.groupby(['segmento', 'clase'])['conteo_total'].sum()
segmentos_dist

segmento  clase
A         1         357
          2         346
          3         399
B         1         686
          2         645
          3         571
C         1         365
          2         516
          3         617
D         1         782
          2         506
          3         395
E         1         254
          2          42
          3           0
S         1        1824
Name: conteo_total, dtype: int64

### Totales de registros por segmento

In [169]:
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum() # Total de registros por segmento
total_segmento

segmento
A    1102
B    1902
C    1498
D    1683
E     296
S    1824
Name: conteo_total, dtype: int64

EN PORCENTAJE

In [170]:
# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()
# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100
# Crear un DataFrame con los resultados
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

print("Total de registros: ",total_registros_total)
#porcentaje_del_total_porsegmentos
resultados

Total de registros:  8305


,segmento,total_por_segmento,porcentaje_del_total
0,A,1102,13.269115
1,B,1902,22.901866
2,C,1498,18.037327
3,D,1683,20.264901
4,E,296,3.564118
5,S,1824,21.962673


### Porcentaje de categoría por segmento

#Obtener el total de registros por segmento
total_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()
#Calcular los porcentajes
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100
#Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje']]
segmentos_percent

In [171]:
# Definir el orden personalizado para segmento y clase
orden_segmento = ['A', 'B', 'C', 'D', 'E', 'S']
orden_clase = ['1', '2', '3']
# Convertir las columnas segmento y clase a categorías con el orden personalizado
segmentos_percent['segmento'] = pd.Categorical(segmentos_percent['segmento'], categories=orden_segmento, ordered=True)
segmentos_percent['clase'] = pd.Categorical(segmentos_percent['clase'], categories=orden_clase, ordered=True)
#segmentos_percent.drop(columns=['categoria'], inplace=True)
# Ordenar el DataFrame según las categorías definidas
segmentos_percent = segmentos_percent.sort_values(by=['segmento', 'clase'])
segmentos_percent
#porcentaje_del_total_porsegmentos

KeyError: 'segmento'

_______________________________
___________
________

In [133]:
porcentaje_del_total_porsegmentos # CONTEO Y % DE REGISTROS POR SEGMENTO, [8305 REGISTROS GRAL]

,segmento,total_por_segmento,porcentaje_del_total
0,A,1102,13.269115
1,B,1902,22.901866
2,C,1498,18.037327
3,D,1683,20.264901
4,E,296,3.564118
5,S,1824,21.962673


In [76]:
segmentos_percent # % CATEGORÍA EN C/SEGMENTO

,segmento,clase,porcentaje
11,A,1,32.395644
12,A,2,31.397459
10,A,3,36.206897
7,B,1,36.067298
8,B,2,33.911672
9,B,3,30.021030
6,C,1,24.365821
5,C,2,34.445928
4,C,3,41.188251
1,D,1,46.464646


- Porcentaje de categoría por segmento [segmentos_percent]
- Total % que ocupa el segmento del df gral porcentaje_del_total_porsegmentos['porcentaje_del_total']

In [134]:
# Asignando los mismos porcientos de acuerdo a los segmentos dados.
#segmento_porciento_gral = [13.269115,13.269115,13.269115,
#                                         22.901866,22.901866,22.901866,
#                                         18.037327,18.037327,18.037327,
#                                         20.264901,20.264901,20.264901,
#                                         3.564118,3.564118,3.564118,
#                                         21.962673,21.962673,21.962673]
resultados = porcentaje_del_total_porsegmentos
A_P = resultados[resultados['segmento'] == 'A']
A_P = A_P['porcentaje_del_total']
B_P = resultados[resultados['segmento'] == 'B']
B_P = B_P['porcentaje_del_total']
C_P = resultados[resultados['segmento'] == 'C']
C_P = C_P['porcentaje_del_total']
D_P = resultados[resultados['segmento'] == 'D']
D_P = D_P['porcentaje_del_total']
E_P = resultados[resultados['segmento'] == 'E']
E_P = E_P['porcentaje_del_total']
S_P = resultados[resultados['segmento'] == 'S']
S_P = S_P['porcentaje_del_total']
segmento_porciento_gral = pd.concat([A_P,A_P,A_P, B_P,B_P,B_P,C_P,C_P,C_P,D_P,D_P,D_P, E_P,E_P,E_P, S_P]) # Combinar los resultados
#segmento_porciento_gral = pd.DataFrame() # Crear nuevo DataFrame
segmento_porciento_gral = pd.Series(segmento_porciento_gral)
segmento_porciento_gral
#segment_per['segmento_porciento_gral'] = [A_P,A_P,A_P, B_P,B_P,B_P,C_P,C_P,C_P,D_P,D_P,D_P, E_P,E_P,E_P, S_P] # Combinar los resultados
#segment_per

df_percent = pd.DataFrame() # Crear nuevo DataFrame
df_percent = pd.concat([segmentos_percent,segmento_porciento_gral ], axis=0,ignore_index=True) #En columna
#df_percent['segmento_porciento_gral'] = [segmento_porciento_gral]
#segmentos_percent.drop(files=['porcentaje_clase_segmento_prop'], inplace=True)
df_percent.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15], axis=0, inplace=True)
df_percent = df_percent.rename(columns={0: 'segmento_porciento_gral'})
df_percent.drop(columns=['segmento','clase','porcentaje'], inplace=True,)
df_percent= pd.concat([segmentos_percent,df_percent.reindex(segmentos_percent.index)], axis=1, ignore_index=False) #En columna
#df_percents = pd.concat([segmentos_percent,df_percent ], axis=0) #En columna

df_percent



,segmento,clase,porcentaje,segmento_porciento_gral
11,A,1,32.395644,NaN
12,A,2,31.397459,NaN
10,A,3,36.206897,NaN
7,B,1,36.067298,NaN
8,B,2,33.911672,NaN
9,B,3,30.021030,NaN
6,C,1,24.365821,NaN
5,C,2,34.445928,NaN
4,C,3,41.188251,NaN
1,D,1,46.464646,NaN


df_segmentos_percent = pd.DataFrame() # Crear el nuevo DataFrame

#df_segmentos_per = pd.concat([resultados, segmentos_percent], axis=0) # Unirlo por columnas
df_segmentos_per = pd.DataFrame() # Crear el nuevo DataFrame

df_edu['Academic_Degree'] = ['Postgrado', 'Universidad', 'Preparatoria', 'Secundaria', 'Otro']
df_edu['Percentage'] = [POST_PERCENT, UNI_PERCENT, PREP_PERCENT, SEC_PERCENT, OTRO_PERCENT]
#Mostrar el nuevo DataFrame
print(df_edu)
13.269115
df_segmentos_per = segmentos_percent[segmentos_percent['segmento'] == 'A']


In [43]:
# segmentos_percent : SEGMENTOS, CLASE, PORCENTAJE
# resultados : segmento, total_por_segmento, porcentaje_del_total 

conteo_total_df = pd.DataFrame(resultados) # CONTEO Y % DE REGISTROS POR SEGMENTOS, [8305 REGISTROS GRAL]
# Crear el DataFrame para el porcentaje de clase
conteo_porcentaje_df = pd.DataFrame(segmentos_percent)  # Datos para el porcentaje de clase, % CATEGORÍA EN C/SEGMENTO

# Calcular el porcentaje que ocupa cada clase de cada segmento en proporción con el porcentaje del total de registros
# % del segmento entre el % del total general
conteo_porcentaje_df['porcentaje_clase_segmento_prop'] = conteo_porcentaje_df['porcentaje'] / conteo_total_df['porcentaje_del_total'] * 100
# Mostrar el DataFrame resultante
print(conteo_porcentaje_df[['segmento', 'clase', 'porcentaje_clase_segmento_prop']])

   segmento clase  porcentaje_clase_segmento_prop
11        A     1                             NaN
12        A     2                             NaN
10        A     3                             NaN
7         B     1                             NaN
8         B     2                             NaN
9         B     3                             NaN
6         C     1                             NaN
5         C     2                      156.838504
4         C     3                     1155.636569
1         D     1                      202.885851
2         D     2                      166.684119
3         D     3                      115.815984
13        E     1                             NaN
14        E     2                             NaN
15        E     3                             NaN
0         S     1                      753.629764


In [98]:
# Crear una lista para almacenar los resultados
resultados_por_segmento = []

# Iterar sobre cada segmento en el DataFrame resultados
for segmento in resultados['segmento']:
    # Obtener el total de registros para el segmento actual
    total_segmento_actual = resultados[resultados['segmento'] == segmento]['total_por_segmento'].values[0]
    # Calcular el porcentaje de cada clase del segmento actual
    porcentaje_clase_actual = conteo_total_df[conteo_total_df['segmento'] == segmento].groupby('clase')['conteo_total'].sum() / total_segmento_actual * 100
    # Crear un DataFrame con los resultados para el segmento actual
    resultado_segmento_actual = pd.DataFrame({
        'segmento': segmento,
        'clase': porcentaje_clase_actual.index,
        'porcentaje_del_segmento': porcentaje_clase_actual.values
    })
    # Agregar el DataFrame del segmento actual a la lista de resultados
    resultados_por_segmento.append(resultado_segmento_actual)

# Concatenar todos los resultados en un solo DataFrame
porcentaje_por_clase_df = pd.concat(resultados_por_segmento)

print(porcentaje_por_clase_df)


  segmento clase  porcentaje_del_segmento
0        A     1                32.395644
1        A     2                31.397459
2        A     3                36.206897
0        B     1                36.067298
1        B     2                33.911672
2        B     3                30.021030
0        C     1                24.365821
1        C     2                34.445928
2        C     3                41.188251
0        D     1                46.464646
1        D     2                30.065359
2        D     3                23.469994
0        E     1                85.810811
1        E     2                14.189189
2        E     3                 0.000000
0        S     1               100.000000


In [95]:
# Crear un DataFrame con los resultados anteriores
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

# Calcular el porcentaje que cada clase ocupa del total de su segmento
porcentaje_por_clase = []
for segmento in resultados['segmento']:
    # Obtener el total de registros para el segmento actual
    total_segmento_actual = resultados[resultados['segmento'] == segmento]['total_por_segmento'].values[0]
    # Calcular el porcentaje de cada clase del segmento actual
    porcentaje_clase_actual = conteo_total_df[conteo_total_df['segmento'] == segmento].groupby('clase')['conteo_total'].sum() / total_segmento_actual * 100
    # Añadir los porcentajes al resultado
    porcentaje_por_clase.append(porcentaje_clase_actual)

# Concatenar los porcentajes de clase en un solo DataFrame
porcentaje_por_clase_df = pd.concat(porcentaje_por_clase)

print(porcentaje_por_clase_df)


clase
1     32.395644
2     31.397459
3     36.206897
1     36.067298
2     33.911672
3     30.021030
1     24.365821
2     34.445928
3     41.188251
1     46.464646
2     30.065359
3     23.469994
1     85.810811
2     14.189189
3      0.000000
1    100.000000
Name: conteo_total, dtype: float64


In [96]:
# Crear una lista para almacenar los resultados
resultados_por_segmento = []

# Iterar sobre cada segmento en el DataFrame resultados
for segmento in resultados['segmento']:
    # Obtener el total de registros para el segmento actual
    total_segmento_actual = resultados[resultados['segmento'] == segmento]['total_por_segmento'].values[0]
    # Calcular el porcentaje de cada clase del segmento actual
    porcentaje_clase_actual = conteo_total_df[conteo_total_df['segmento'] == segmento].groupby('clase')['conteo_total'].sum() / total_segmento_actual * 100
    # Crear un DataFrame con los resultados para el segmento actual
    resultado_segmento_actual = pd.DataFrame({
        'segmento': segmento,
        'clase': porcentaje_clase_actual.index,
        'porcentaje_del_segmento': porcentaje_clase_actual.values
    })
    # Agregar el DataFrame del segmento actual a la lista de resultados
    resultados_por_segmento.append(resultado_segmento_actual)

# Concatenar todos los resultados en un solo DataFrame
porcentaje_por_clase_df = pd.concat(resultados_por_segmento)

print(porcentaje_por_clase_df)


  segmento clase  porcentaje_del_segmento
0        A     1                32.395644
1        A     2                31.397459
2        A     3                36.206897
0        B     1                36.067298
1        B     2                33.911672
2        B     3                30.021030
0        C     1                24.365821
1        C     2                34.445928
2        C     3                41.188251
0        D     1                46.464646
1        D     2                30.065359
2        D     3                23.469994
0        E     1                85.810811
1        E     2                14.189189
2        E     3                 0.000000
0        S     1               100.000000


### 

In [94]:
import pandas as pd

# Definir el orden personalizado para segmento y clase
orden_segmento = ['A', 'B', 'C', 'D', 'E', 'S']
orden_clase = ['1', '2', '3']

# Convertir las columnas segmento y clase a categorías con el orden personalizado
segmentos_percent['segmento'] = pd.Categorical(segmentos_percent['segmento'], categories=orden_segmento, ordered=True)
segmentos_percent['clase'] = pd.Categorical(segmentos_percent['clase'], categories=orden_clase, ordered=True)

# Calcular el porcentaje del porcentaje total de todos los registros
segmentos_percent['porcentaje_del_total'] = (segmentos_percent['porcentaje'] / total_registros_total) * 100

# Ordenar el DataFrame según las categorías definidas
segmentos_percent = segmentos_percent.sort_values(by=['segmento', 'clase','porcentaje', 'porcentaje_del_total'])

print(segmentos_percent)


   segmento clase  porcentaje  porcentaje_del_total
11        A     1   32.395644              0.390074
12        A     2   31.397459              0.378055
10        A     3   36.206897              0.435965
7         B     1   36.067298              0.434284
8         B     2   33.911672              0.408328
9         B     3   30.021030              0.361481
6         C     1   24.365821              0.293387
5         C     2   34.445928              0.414761
4         C     3   41.188251              0.495945
1         D     1   46.464646              0.559478
2         D     2   30.065359              0.362015
3         D     3   23.469994              0.282601
13        E     1   85.810811              1.033243
14        E     2   14.189189              0.170851
15        E     3    0.000000              0.000000
0         S     1  100.000000              1.204094


In [82]:
# Calcular el total de todos los registros
total_registros_total = conteo_total_df['conteo_total'].sum()
print(total_registros_total)
# Calcular los porcentajes de los segmentos
segmentos_percent = conteo_total_df.merge(total_segmento, on='segmento', suffixes=('', '_total'))
segmentos_percent['porcentaje'] = segmentos_percent['conteo_total'] / segmentos_percent['conteo_total_total'] * 100

# Calcular el porcentaje del porcentaje total de todos los registros
segmentos_percent['porcentaje_del_total'] = (segmentos_percent['porcentaje'] / total_registros_total) * 100

# Seleccionar las columnas necesarias y eliminar las columnas auxiliares
segmentos_percent = segmentos_percent[['segmento', 'clase', 'porcentaje', 'porcentaje_del_total']]

print(segmentos_percent)


8305
   segmento clase  porcentaje  porcentaje_del_total
0         S     1  100.000000              1.204094
1         D     1   46.464646              0.559478
2         D     2   30.065359              0.362015
3         D     3   23.469994              0.282601
4         C     3   41.188251              0.495945
5         C     2   34.445928              0.414761
6         C     1   24.365821              0.293387
7         B     1   36.067298              0.434284
8         B     2   33.911672              0.408328
9         B     3   30.021030              0.361481
10        A     3   36.206897              0.435965
11        A     1   32.395644              0.390074
12        A     2   31.397459              0.378055
13        E     1   85.810811              1.033243
14        E     2   14.189189              0.170851
15        E     3    0.000000              0.000000


In [103]:
# Calcular el porcentaje que ocupa cada clase del total de registros en su segmento
conteo_total_df['porcentaje_clase_segmento'] = conteo_total_df.groupby(['segmento', 'clase'])['conteo_total'].transform(lambda x: x / x.sum() * 100).fillna(0)

# Calcular el porcentaje que ocupa cada segmento del total de registros
total_registros_total = conteo_total_df['conteo_total'].sum()
conteo_total_df['porcentaje_segmento'] = conteo_total_df.groupby('segmento')['conteo_total'].transform(lambda x: x.sum() / total_registros_total * 100).fillna(0)

# Calcular el porcentaje que ocupa cada clase de cada segmento en relación con el porcentaje que ocupan esos registros del 100% del segmento
conteo_total_df['porcentaje_clase_segmento_rel'] = conteo_total_df['porcentaje_clase_segmento'] / conteo_total_df['porcentaje_segmento']

# Ordenar el DataFrame según el orden personalizado para segmento y clase
conteo_total_df['segmento'] = pd.Categorical(conteo_total_df['segmento'], categories=orden_segmento, ordered=True)
conteo_total_df['clase'] = pd.Categorical(conteo_total_df['clase'], categories=orden_clase, ordered=True)
conteo_total_df = conteo_total_df.sort_values(by=['segmento', 'clase'])

# Mostrar el DataFrame resultante
print(conteo_total_df[['segmento', 'clase', 'porcentaje_clase_segmento_rel']])


   segmento clase  porcentaje_clase_segmento_rel
11        A     1                       7.536298
12        A     2                       7.536298
9         A     3                       7.536298
3         B     1                       4.366456
4         B     2                       4.366456
5         B     3                       4.366456
10        C     1                       5.544059
7         C     2                       5.544059
2         C     3                       5.544059
1         D     1                       4.934641
6         D     2                       4.934641
8         D     3                       4.934641
13        E     1                      28.057432
14        E     2                      28.057432
15        E     3                       0.000000
0         S     1                       4.553180


In [119]:
import pandas as pd

# Lista de DataFrames y sus nombres correspondientes
dataframes_list = [
    segment_df_alfa_q_oct_2022_pachuca,
    segment_df_alfa_q_nov_2022_pachuca,
    segment_df_alfa_q_feb_2023_pachuca,
    segment_df_alfa_q_mar_2023_pachuca,
    segment_df_alfa_q_jun_2023_pachuca,
    segment_df_alfa_q_jul_2023_pachuca
]

# Inicializar un diccionario para almacenar el conteo total de valores únicos
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])

# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Crear una nueva fila como un diccionario de Python
nueva_fila = {'conteo_total': 0, 'segmento': 'E', 'clase': '3'}
# Agregar la nueva fila al DataFrame
conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)

# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()

# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()

# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100

# Calcular el porcentaje que ocupa cada clase de cada segmento en proporción con el total de registros en todos los segmentos
conteo_total_df['porcentaje_clase_segmento_con_total'] = conteo_total_df['conteo_total'] / total_registros_total * 100

# Crear un DataFrame con los resultados de los totales por segmento y su porcentaje del total
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

# Mostrar los resultados
print("Total de registros por segmento y su porcentaje del total:")
print(resultados)

print("\nConteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:")
print(conteo_total_df)


Total de registros por segmento y su porcentaje del total:
  segmento  total_por_segmento  porcentaje_del_total
0        A                1102             13.269115
1        B                1902             22.901866
2        C                1498             18.037327
3        D                1683             20.264901
4        E                 296              3.564118
5        S                1824             21.962673

Conteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:
    conteo_total segmento clase  porcentaje_clase_segmento_con_total
0           1824        S     1                            21.962673
1            782        D     1                             9.416014
2            617        C     3                             7.429259
3            686        B     1                             8.260084
4            645        B     2                             7.766406
5            571        B     3                             6.8

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13160\829808386.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)


In [128]:
import pandas as pd

# Definir el orden personalizado para segmento y clase
orden_segmento = ['A', 'B', 'C', 'D', 'E', 'S']
orden_clase = ['1', '2', '3']

# Lista de DataFrames y sus nombres correspondientes
dataframes_list = [
    segment_df_alfa_q_oct_2022_pachuca,
    segment_df_alfa_q_nov_2022_pachuca,
    segment_df_alfa_q_feb_2023_pachuca,
    segment_df_alfa_q_mar_2023_pachuca,
    segment_df_alfa_q_jun_2023_pachuca,
    segment_df_alfa_q_jul_2023_pachuca
]

# Inicializar un diccionario para almacenar el conteo total de valores únicos
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])

# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Crear una nueva fila como un diccionario de Python
nueva_fila = {'conteo_total': 0, 'segmento': 'E', 'clase': '3'}
# Agregar la nueva fila al DataFrame
conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)

# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()

# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()

# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100

# Calcular el porcentaje que ocupa cada clase de cada segmento en proporción con el total de registros en todos los segmentos
conteo_total_df['porcentaje_clase_segmento_con_total'] = conteo_total_df['conteo_total'] / total_registros_total * 100

# Crear un DataFrame con los resultados de los totales por segmento y su porcentaje del total
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

# Ordenar el DataFrame de conteo total según las categorías definidas
conteo_total_df['segmento'] = pd.Categorical(conteo_total_df['segmento'], categories=orden_segmento, ordered=True)
conteo_total_df['clase'] = pd.Categorical(conteo_total_df['clase'], categories=orden_clase, ordered=True)
conteo_total_df = conteo_total_df.sort_values(by=['segmento', 'clase'])

# Mostrar los resultados
print("Total de registros por segmento y su porcentaje del total:")
print(resultados)

print("\nConteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:")
conteo_total_df







Total de registros por segmento y su porcentaje del total:
  segmento  total_por_segmento  porcentaje_del_total
0        A                1102             13.269115
1        B                1902             22.901866
2        C                1498             18.037327
3        D                1683             20.264901
4        E                 296              3.564118
5        S                1824             21.962673

Conteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13160\576484756.py:39: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,conteo_total,segmento,clase,porcentaje_clase_segmento_con_total
11,357,A,1,4.298615
12,346,A,2,4.166165
9,399,A,3,4.804335
3,686,B,1,8.260084
4,645,B,2,7.766406
5,571,B,3,6.875376
10,365,C,1,4.394943
7,516,C,2,6.213125
2,617,C,3,7.429259
1,782,D,1,9.416014


In [132]:
import pandas as pd
import plotly.graph_objects as go

# Definir el orden personalizado para segmento y clase
orden_segmento = ['A', 'B', 'C', 'D', 'E', 'S']
orden_clase = ['1', '2', '3']

# Lista de DataFrames y sus nombres correspondientes
dataframes_list = [
    segment_df_alfa_q_oct_2022_pachuca,
    segment_df_alfa_q_nov_2022_pachuca,
    segment_df_alfa_q_feb_2023_pachuca,
    segment_df_alfa_q_mar_2023_pachuca,
    segment_df_alfa_q_jun_2023_pachuca,
    segment_df_alfa_q_jul_2023_pachuca
]

# Inicializar un diccionario para almacenar el conteo total de valores únicos
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])

# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Crear una nueva fila como un diccionario de Python
nueva_fila = {'conteo_total': 0, 'segmento': 'E', 'clase': '3'}
# Agregar la nueva fila al DataFrame
conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)

# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()

# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()

# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100

# Crear un DataFrame con los resultados de los totales por segmento y su porcentaje del total
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

# Ordenar el DataFrame de conteo total según las categorías definidas
conteo_total_df['segmento'] = pd.Categorical(conteo_total_df['segmento'], categories=orden_segmento, ordered=True)
conteo_total_df['clase'] = pd.Categorical(conteo_total_df['clase'], categories=orden_clase, ordered=True)
conteo_total_df = conteo_total_df.sort_values(by=['segmento', 'clase'])

# Mostrar los resultados
print("Total de registros por segmento y su porcentaje del total:")
print(resultados)

print("\nConteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:")
print(conteo_total_df)

# Crear una figura para el gráfico
fig = go.Figure()

# Iterar sobre los segmentos y clases en el DataFrame de resultados
for segmento in orden_segmento:
    # Filtrar el DataFrame de resultados para el segmento actual
    df_segmento = resultados[resultados['segmento'] == segmento]
    
    # Añadir barras para cada clase dentro del segmento
    fig.add_trace(go.Bar(
        x=[f'{segmento}-{clase}' for clase in orden_clase],  # Nombre de las barras
        y=df_segmento['porcentaje_del_total'],  # Porcentajes de cada clase
        name=segmento  # Nombre del segmento
    ))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Porcentaje de cada clase de cada segmento en relación con el total de registros',
    xaxis_title='Clase',
    yaxis_title='Porcentaje (%)',
    barmode='stack'  # Apilar las barras
)

# Mostrar el gráfico
fig.show()


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13160\722907212.py:40: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



Total de registros por segmento y su porcentaje del total:
  segmento  total_por_segmento  porcentaje_del_total
0        A                1102             13.269115
1        B                1902             22.901866
2        C                1498             18.037327
3        D                1683             20.264901
4        E                 296              3.564118
5        S                1824             21.962673

Conteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:
    conteo_total segmento clase
11           357        A     1
12           346        A     2
9            399        A     3
3            686        B     1
4            645        B     2
5            571        B     3
10           365        C     1
7            516        C     2
2            617        C     3
1            782        D     1
6            506        D     2
8            395        D     3
13           254        E     1
14            42        E    

In [134]:
import pandas as pd
import plotly.graph_objects as go

# Definir el orden personalizado para segmento y clase
orden_segmento = ['A', 'B', 'C', 'D', 'E', 'S']
orden_clase = ['1', '2', '3']

# Lista de DataFrames y sus nombres correspondientes
dataframes_list = [
    segment_df_alfa_q_oct_2022_pachuca,
    segment_df_alfa_q_nov_2022_pachuca,
    segment_df_alfa_q_feb_2023_pachuca,
    segment_df_alfa_q_mar_2023_pachuca,
    segment_df_alfa_q_jun_2023_pachuca,
    segment_df_alfa_q_jul_2023_pachuca
]

# Inicializar un diccionario para almacenar el conteo total de valores únicos
conteo_total = {}

# Iterar sobre la lista de DataFrames
for df in dataframes_list:
    # Calcular el conteo de valores únicos de 'categoria' en el DataFrame actual
    conteo_actual = df['categoria'].value_counts().to_dict()
    # Agregar el conteo actual al conteo total
    for categoria, conteo in conteo_actual.items():
        conteo_total[categoria] = conteo_total.get(categoria, 0) + conteo

# Convertir el diccionario a un DataFrame
conteo_total_df = pd.DataFrame(list(conteo_total.items()), columns=['categoria', 'conteo_total'])

# Añadir una nuevas columnas que extraigan el número y letra de la categoría
conteo_total_df['segmento'] = conteo_total_df['categoria'].str[0]
conteo_total_df['clase'] = conteo_total_df['categoria'].str.extract(r'(\d+)').fillna('1')
conteo_total_df.drop(columns=['categoria'], inplace=True)

# Crear una nueva fila como un diccionario de Python
nueva_fila = {'conteo_total': 0, 'segmento': 'E', 'clase': '3'}
# Agregar la nueva fila al DataFrame
conteo_total_df = conteo_total_df.append(nueva_fila, ignore_index=True)

# Calcular el total de registros por cada segmento, incluyendo todas sus categorías
total_por_segmento = conteo_total_df.groupby('segmento')['conteo_total'].sum()

# Calcular el total de registros en todos los segmentos
total_registros_total = total_por_segmento.sum()

# Calcular el porcentaje que representa cada total de segmento con respecto al total de registros en todo el DataFrame
porcentaje_total_segmento = (total_por_segmento / total_registros_total) * 100

# Calcular el porcentaje que ocupa cada clase de cada segmento en proporción con el total de registros en todos los segmentos
conteo_total_df['porcentaje_clase_segmento_con_total'] = conteo_total_df['conteo_total'] / total_registros_total * 100

# Crear un DataFrame con los resultados de los totales por segmento y su porcentaje del total
resultados = pd.DataFrame({
    'segmento': total_por_segmento.index,
    'total_por_segmento': total_por_segmento.values,
    'porcentaje_del_total': porcentaje_total_segmento.values
})

# Ordenar el DataFrame de conteo total según las categorías definidas
conteo_total_df['segmento'] = pd.Categorical(conteo_total_df['segmento'], categories=orden_segmento, ordered=True)
conteo_total_df['clase'] = pd.Categorical(conteo_total_df['clase'], categories=orden_clase, ordered=True)
conteo_total_df = conteo_total_df.sort_values(by=['segmento', 'clase'])

# Mostrar los resultados
print("Total de registros por segmento y su porcentaje del total:")
print(resultados)

print("\nConteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:")
print(conteo_total_df)

# Crear una figura
fig = go.Figure()

# Iterar sobre los segmentos y clases en el DataFrame de conteo total
for segmento in orden_segmento:
    for clase in orden_clase:
        # Filtrar el DataFrame para el segmento y la clase actual
        df_clase = conteo_total_df[(conteo_total_df['segmento'] == segmento) & (conteo_total_df['clase'] == clase)]
        
        # Añadir una barra a la figura para la clase actual
        fig.add_trace(go.Bar(
            x=[f'{segmento}-{clase}'],  # Nombre de la barra
            y=[df_clase['porcentaje_clase_segmento_con_total'].values[0]],  # Obtener el porcentaje de la clase
            name=f'{segmento}-{clase}'  # Nombre de la leyenda
        ))

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Porcentaje de cada clase de cada segmento en relación con el total de registros',
    xaxis_title='Clase',
    yaxis_title='Porcentaje (%)',
    barmode='stack'  # Apilar las barras
)

# Mostrar el gráfico
fig.show()


Total de registros por segmento y su porcentaje del total:
  segmento  total_por_segmento  porcentaje_del_total
0        A                1102             13.269115
1        B                1902             22.901866
2        C                1498             18.037327
3        D                1683             20.264901
4        E                 296              3.564118
5        S                1824             21.962673

Conteo total de valores únicos por clase y segmento, y porcentaje en relación con el total:
    conteo_total segmento clase  porcentaje_clase_segmento_con_total
11           357        A     1                             4.298615
12           346        A     2                             4.166165
9            399        A     3                             4.804335
3            686        B     1                             8.260084
4            645        B     2                             7.766406
5            571        B     3                             6.8

C:\Users\yoe11\AppData\Local\Temp\ipykernel_13160\1672527360.py:40: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



IndexError: index 0 is out of bounds for axis 0 with size 0